# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

### Importar o dataset

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [4]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [7]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])

[0 0 0 ... 0 1 0]


In [8]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [111]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

# Feature Scaling

In [112]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [113]:
np.shape(X_train)

(8000, 12)

In [114]:
X_train

array([[-0.996257  , -0.58601101,  1.74954671, ..., -1.5482337 ,
        -1.02429504, -1.40114775],
       [ 1.00375706, -0.58601101, -0.57157662, ...,  0.64589732,
        -1.02429504,  1.6776678 ],
       [ 1.00375706, -0.58601101, -0.57157662, ...,  0.64589732,
         0.97628121,  1.56314435],
       ...,
       [ 1.00375706, -0.58601101, -0.57157662, ...,  0.64589732,
        -1.02429504, -1.51811653],
       [ 1.00375706, -0.58601101, -0.57157662, ..., -1.5482337 ,
         0.97628121,  1.26396131],
       [-0.996257  ,  1.70645259, -0.57157662, ...,  0.64589732,
        -1.02429504, -1.08851177]])

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [115]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [122]:
ann.add(tf.keras.layers.Dense(units=9, activation='elu'))

### Adding the second hidden layer

In [123]:
ann.add(tf.keras.layers.Dense(units=9, activation='elu'))

### Adding the output layer

In [124]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the ANN

In [125]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Part 3 - Training the ANN

### Training the ANN on the Training set

In [127]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 150)

Epoch 1/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3148 - accuracy: 0.8702
Epoch 2/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3156 - accuracy: 0.8729
Epoch 3/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3152 - accuracy: 0.8700
Epoch 4/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3140 - accuracy: 0.8720
Epoch 5/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3145 - accuracy: 0.8717
Epoch 6/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3141 - accuracy: 0.8719
Epoch 7/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3150 - accuracy: 0.8710
Epoch 8/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3140 - accuracy: 0.8704
Epoch 9/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3138 - accuracy: 0.8701
Epoch 10/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3140 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.3135 - accuracy: 0.8730
Epoch 82/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3138 - accuracy: 0.8736
Epoch 83/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3137 - accuracy: 0.8726
Epoch 84/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3131 - accuracy: 0.8736
Epoch 85/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3126 - accuracy: 0.8731
Epoch 86/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3127 - accuracy: 0.8730
Epoch 87/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3130 - accuracy: 0.8734
Epoch 88/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3133 - accuracy: 0.8739
Epoch 89/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3123 - accuracy: 0.8734
Epoch 90/150
250/250 [==============================] - 0s 1ms/step - loss: 0.3137 - accuracy: 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [128]:
y_pred = ann.predict(X_test)
y_pred

array([[0.03116068],
       [0.0374501 ],
       [0.0718241 ],
       ...,
       [0.0242115 ],
       [0.04457402],
       [0.06158975]], dtype=float32)

In [129]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [130]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1507   88]
 [ 217  188]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)